In [1]:
#import antigravity
import pandas as pd
import math
from typing import List
from datetime import datetime
from sqlalchemy import MetaData, Table, create_engine, text
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date

In [2]:
import sys
import os

In [3]:
engine = create_engine('sqlite:///pare.db')

In [4]:
sys.path.append(os.path.join(os.path.dirname('__file__'), 'models'))
sys.path.append(os.path.join(os.path.dirname('__file__'), 'scripts'))
sys.path.append(os.path.join(os.path.dirname('__file__'), 'config'))

In [5]:
from config.params import DEBUG

In [6]:
from scripts.utils import random_item, random_date, random_date_between, random_int, random_float, years_between

In [7]:
from models.sale import Sale, TYPE_LIST
from models.partner import Partner

In [8]:
from scripts.constants import (JOINED_START_DATE, 
                                JOINED_END_DATE,)

joined_start_date = JOINED_START_DATE
joined_end_date = JOINED_END_DATE

In [9]:
partner_df = pd.read_sql('partners', con=engine)
partner_list = partner_df.to_dict(orient='records')

In [10]:
def calculate_total(qty: int, unit_price: float)-> float:
    return qty * unit_price

In [11]:
#random_int(6752, 500000)

In [12]:
calculate_total(198, 0)

0

In [13]:
def generate_sales():
    sales_list = []
    
    for i in range(6500):
        r_type = random_item(TYPE_LIST)
        min_price_unit= r_type['min_price_unit']
        max_price_unit= r_type['max_price_unit']
        min_qty= r_type['min_qty']
        max_qty= r_type['max_qty']
        price_unit= random_float(min_price_unit, max_price_unit)
        qty= random_int(min_qty, max_qty)
        unit_price=price_unit
        total=calculate_total(qty, unit_price)
        partner = random_item(partner_list)
        
        #print(total)
        sales_list.append( 
            Sale(
            id=i,
            partner_id=partner['id'],
            partner_name=partner['name'],
            type_of_product = r_type['name'],
            date=random_date_between(joined_start_date, joined_end_date),
            qty=qty,
            currency=r_type['currency'],
            unit=r_type['unit'],
            unit_price=unit_price,
            total=total,
            total_usd = total /131.5
        )
        )
    return sales_list

In [14]:
sales_list = generate_sales()

In [15]:
sales_list = [sale.dict() for sale in sales_list]

In [16]:
sale_df = pd.DataFrame(sales_list)

In [17]:
sale_df.head()

,id,partner_id,participant_id,partner_name,type_of_product,date,currency,qty,unit_price,unit,total,total_usd
0,0,57,None,Solution Boutique Agricole,Lait de vache,2024-06-19,HTG,15,548.951153,gallon,8.234267e+03,62.618002
1,1,13,None,THE CHOU FERME,Lait de vache,2027-06-22,HTG,7,490.707272,gallon,3.434951e+03,26.121300
2,2,212,None,Mouvement des Investisseurs de Camp-perrin,Mouton entier,2025-12-21,HTG,9,20027.046101,animal,1.802434e+05,1370.672357
3,3,39,None,AGROTECH,Volaille,2025-12-29,HTG,3230,1791.315227,animal,5.785948e+06,43999.605954
4,4,221,None,Ferme La Bellevue,Bovin entier,2026-05-22,HTG,6,344677.908208,animal,2.068067e+06,15726.748663


In [18]:
sum(sale_df['total_usd'])

71144467.76206161

In [19]:
#sale_df.to_csv("./outputs/sales_1.csv")

In [20]:
#sale_df.to_csv("./outputs/sales_2.csv")

In [21]:
#sale_df.to_csv("./outputs/sales_3.csv")

In [22]:
#sale_df.to_csv("./outputs/sales_4.csv")

In [23]:
#sale_df.to_csv("./outputs/sales_5.csv")

In [39]:
#sale_df.to_csv("./outputs/sales_8_final_enterprise.csv")

In [31]:
df = pd.read_csv("./outputs/sales_8_final_enterprise.csv")

In [26]:
#df['total_usd'] = [total / 131.5 for total in df['total']]

In [32]:
df.head()

,Unnamed: 0,id,partner_id,participant_id,partner_name,type_of_product,date,currency,qty,unit_price,unit,total,total_usd
0,0,0,57,NaN,Solution Boutique Agricole,Lait de vache,2024-06-19,HTG,15,548.951153,gallon,8.234267e+03,62.618002
1,1,1,13,NaN,THE CHOU FERME,Lait de vache,2027-06-22,HTG,7,490.707272,gallon,3.434951e+03,26.121300
2,2,2,212,NaN,Mouvement des Investisseurs de Camp-perrin,Mouton entier,2025-12-21,HTG,9,20027.046101,animal,1.802434e+05,1370.672357
3,3,3,39,NaN,AGROTECH,Volaille,2025-12-29,HTG,3230,1791.315227,animal,5.785948e+06,43999.605954
4,4,4,221,NaN,Ferme La Bellevue,Bovin entier,2026-05-22,HTG,6,344677.908208,animal,2.068067e+06,15726.748663


In [28]:
#sum(df['total']) / 131.750

In [40]:
df.groupby(["type_of_product"]).qty.agg(['count', 'min', 'max', 'sum'])
#df.shape

,count,min,max,sum
type_of_product,,,,
Bovin entier,828,5,15,8161
Lait de vache,796,5,50,22283
Mouton entier,794,5,25,11650
Oeufs,806,503,9970,4341164
Viande de bovin,801,26,7496,3177245
Viande de chèvre,830,15,3500,1471044
Viande de mouton,817,20,2496,985142
Volaille,828,23,5000,2047421


In [36]:
df.groupby(["type_of_product"]).unit_price.agg(['min', 'max'])

,min,max
type_of_product,,
Bovin entier,75029.530417,349837.889932
Lait de vache,450.013250,549.967147
Mouton entier,17516.011372,39988.650080
Oeufs,150.000682,199.996929
Viande de bovin,550.088227,699.596494
Viande de chèvre,650.391939,799.908568
Viande de mouton,570.109120,699.651698
Volaille,750.312261,1999.028437


In [38]:
#df.groupby(["partner_name", "type_of_product"]).total_usd.agg(['count', 'min', 'max', 'sum'])